In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns # plotting, correlation
import matplotlib.pyplot as plt # plotting
from sklearn import linear_model # Linear Regression


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## First Read all the datasets

In [ ]:
train = pd.read_csv('/kaggle/input/tabular-playground-series-jul-2021/train.csv', parse_dates = ['date_time'])
test = pd.read_csv('/kaggle/input/tabular-playground-series-jul-2021/test.csv', parse_dates = ['date_time'])
sample_submission = pd.read_csv('/kaggle/input/tabular-playground-series-jul-2021/sample_submission.csv', parse_dates = ['date_time'])

### Get an overview of the data (train)

In [ ]:
train.head(2)

### Get overview of our testing dataset (test)

In [ ]:
test.head(2)

### Get overview of what our sample submission should look like

In [ ]:
sample_submission.head(2)

### Check for missing values

In [ ]:
train.isnull().sum()

## EDA

In [ ]:
train.set_index('date_time')

### Check the correlation among different variables 

The Pearson and Spearman correlation gives the linear and the rank correlation among the variables, respectively. 

In [ ]:
sns.set(rc={'figure.figsize':(11.7,8.27)})
sns.heatmap(train.corr('pearson'), annot=True)

In [ ]:
sns.heatmap(train.corr('spearman'), annot=True)

All the three of our outcome variables have strong correlation with sensors 1 to sensors 5 and very negligeable correlation with the temperature (deg_C), relative and absolute humidity. Also 'target_benzene' has a strong linear correlation with sensor 3 compared to the others. 

For now lets leave it at that. But probably we can use it to do much better feature selection and avoid multicollinearity problems.


In [ ]:
sns.pairplot(train)

## Simple Linear Regression Fit

In [ ]:
predictors = ['deg_C', 'relative_humidity', 'absolute_humidity', 'sensor_1', 'sensor_2', 'sensor_3', 'sensor_4', 'sensor_5']
objective_1 = 'target_carbon_monoxide'
objective_2 = 'target_benzene'
objective_3 = 'target_nitrogen_oxides'

In [ ]:
X = train[predictors].values
y_pred={objective_1:[], objective_2:[], objective_3:[]}
for outcome in [objective_1, objective_2, objective_3]:
    y = train[outcome].values.reshape(-1,1)
    regr = linear_model.LinearRegression()
    regr.fit(X, y)
    y_pred[outcome] = regr.predict(test.set_index('date_time')).reshape(test.shape[0],)

In [ ]:
df_submit = pd.DataFrame.from_dict(y_pred) 
df_submit.insert(0, 'date_time', test['date_time'])
df_submit